# Prelim

Notebook for ArchR preprocessing 

The results in the notebook were genreated using the 10X PBMC ATAC dataset: https://support.10xgenomics.com/single-cell-atac/datasets/1.2.0/atac_pbmc_10k_nextgem

<b>ArchR installation </b>

Install from [https://github.com/settylab/ArchR](https://github.com/settylab/ArchR)

```
library(devtools)
devtools::install_github("GreenleafLab/ArchR", ref="master", repos = BiocManager::repositories())
```

Update your ArchR with the customized version
```
R CMD INSTALL -l <PATH to R personal library> <path to Git clone >
```

Review the notebook `PBMC-RNA-standalone.ipynb` for setup instructions.
Install MACS2 for peak celling 
```
conda install -c bioconda macs2 
```



## Load data

Following files are required for using these tools:
1. ATAC fragments file. [Example](https://fh-pi-setty-m-eco-public.s3.us-west-2.amazonaws.com/single-cell-primers/scatac/atac_pbmc_10k_nextgem_fragments.tsv.gz)
2. Index for the fragments file. [Example](https://fh-pi-setty-m-eco-public.s3.us-west-2.amazonaws.com/single-cell-primers/scatac/atac_pbmc_10k_nextgem_fragments.tsv.gz.tbi)
3. Per barcode metrics. [Example](https://fh-pi-setty-m-eco-public.s3.us-west-2.amazonaws.com/single-cell-primers/scatac/atac_pbmc_10k_nextgem_singlecell.csv)

Use the above files to run ArchR using the ArchR preprocessing script: https://github.com/dpeerlab/SEACells/blob/main/notebooks/ArchR/ArchR-preprocessing-nfr-peaks.R


# Imports

In [1]:
import os
import pandas as pd
import numpy as np

import scanpy as sc
import pyranges as pr
import warnings

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import SEACells

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


# Load data

This section loads all the results from ArchR to generate an Anndata object

## ATAC

In [6]:
data_dir = os.path.expanduser('/wynton/group/pollen/jding/brainchromatin/multiome/ArchR/seacells_nfr/export/')

In [4]:
data_dir = os.path.expanduser('/wynton/group/pollen/jding/brainchromatin/multiome/ArchR/seacells/export/')

Load all the exported results from ArchR

### Peaks data

In [5]:
# Peaks data
from scipy.io import mmread
counts = mmread(data_dir + 'peak_counts/counts.mtx')

In [6]:
# Cell and peak information
cells = pd.read_csv(data_dir + 'peak_counts/cells.csv', index_col=0).iloc[:, 0]
peaks = pd.read_csv(data_dir + 'peak_counts/peaks.csv', index_col=0)
peaks.index = peaks['seqnames'] + ':' + peaks['start'].astype(str) + '-' + peaks['end'].astype(str)
peaks.head()

,seqnames,start,end,width,strand,score,replicateScoreQuantile,groupScoreQuantile,Reproducibility,GroupReplicate,nearestGene,distToGeneStart,peakType,distToTSS,nearestTSS,GC,idx,N
chr1:817083-817583,chr1,817083,817583,501,*,11.16000,0.911,0.702,2,C19._.Rep2,FAM87B,38,Promoter,37,uc057aum.1,0.4770,1,0
chr1:826707-827207,chr1,826707,827207,501,*,2.23785,0.221,0.044,2,C3._.hft_ctx_w21_dc2r2_r2,LINC01128,1819,Exonic,124,uc057auo.1,0.6088,2,0
chr1:827277-827777,chr1,827277,827777,501,*,77.40670,0.899,0.856,2,C6._.hft_ctx_w21_dc2r2_r2,LINC01128,2389,Promoter,4,uc057auo.1,0.6866,3,0
chr1:858594-859094,chr1,858594,859094,501,*,5.96004,0.365,0.180,2,C12._.hft_ctx_w21_dc2r2_r1,LINC01128,33706,Exonic,7495,uc057auo.1,0.5369,4,0
chr1:869643-870143,chr1,869643,870143,501,*,46.77340,0.792,0.727,3,C13._.hft_ctx_w21_dc1r3_r1,FAM41C,1822,Intronic,307,uc057aux.1,0.7226,5,0


In [7]:
ad = sc.AnnData(counts.T)
ad.obs_names = cells
ad.var_names = peaks.index
for col in peaks.columns:
    ad.var[col] = peaks[col]

/scratch/jding/ipykernel_23211/1812160205.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  ad = sc.AnnData(counts.T)


In [8]:
ad.X = ad.X.tocsr()

In [9]:
ad

AnnData object with n_obs × n_vars = 8286 × 260363
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'nearestGene', 'distToGeneStart', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'

### SVD

In [10]:
ad.obsm['X_svd'] = pd.read_csv(data_dir + 'svd.csv', index_col=0).loc[ad.obs_names, : ].values

### Metadata

In [11]:
cell_meta = pd.read_csv(data_dir + 'cell_metadata.csv', index_col=0).loc[ad.obs_names, : ]
for col in cell_meta.columns:
    ad.obs[col] = cell_meta[col].values

In [12]:
ad

AnnData object with n_obs × n_vars = 8286 × 260363
    obs: 'Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'Clusters', 'ReadsInPeaks', 'FRIP'
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'nearestGene', 'distToGeneStart', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'
    obsm: 'X_svd'

### Gene scores

In [13]:
# Gene scores
gene_scores = pd.read_csv(data_dir + 'gene_scores.csv', index_col=0).T

In [14]:
ad.obsm['GeneScores'] = gene_scores.loc[ad.obs_names, :].values
ad.uns['GeneScoresColums'] = gene_scores.columns.values

In [15]:
ad

AnnData object with n_obs × n_vars = 8286 × 260363
    obs: 'Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'Clusters', 'ReadsInPeaks', 'FRIP'
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'nearestGene', 'distToGeneStart', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'
    uns: 'GeneScoresColums'
    obsm: 'X_svd', 'GeneScores'

# Visualizations

In [16]:
# Leiden and UMAP
warnings.filterwarnings('ignore')
sc.pp.neighbors(ad, use_rep='X_svd')
sc.tl.umap(ad)
sc.tl.leiden(ad)
warnings.filterwarnings('default')

In [19]:
sc.pl.scatter(ad, basis='umap', color=['celltype', 'phenograph'])

ValueError: key 'phenograph' is invalid! pass valid observation annotation, one of ['Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'celltype', 'Clusters', 'ReadsInPeaks', 'FRIP', 'leiden'] or a gene name Index(['chr1:827236-827736', 'chr1:869647-870147', 'chr1:904511-905011',
       'chr1:905189-905689', 'chr1:909918-910418', 'chr1:912789-913289',
       'chr1:916484-916984', 'chr1:919549-920049', 'chr1:920980-921480',
       'chr1:923600-924100',
       ...
       'chrX:155778767-155779267', 'chrX:155820024-155820524',
       'chrX:155845178-155845678', 'chrX:155871832-155872332',
       'chrX:155881028-155881528', 'chrX:155891826-155892326',
       'chrX:155905550-155906050', 'chrX:155910246-155910746',
       'chrX:155941897-155942397', 'chrX:155956603-155957103'],
      dtype='object', length=243018)

# Save

 Save the anndata object for downstream usage

In [20]:
ad

AnnData object with n_obs × n_vars = 8181 × 243018
    obs: 'Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'celltype', 'Clusters', 'ReadsInPeaks', 'FRIP', 'leiden'
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'nearestGene', 'distToGeneStart', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'
    uns: 'GeneScoresColums', 'neighbors', 'umap', 'leiden'
    obsm: 'X_svd', 'GeneScores', 'X_umap'
    obsp: 'distances', 'connectivities'

In [17]:
ad.write(data_dir + 'archr.h5ad')